In [37]:
from openpyxl import load_workbook

In [5]:
from pathlib import Path
from typing import Optional, Tuple, List
from openpyxl import load_workbook
from openpyxl.worksheet.worksheet import Worksheet
from openpyxl.utils.cell import range_boundaries

In [40]:
def get_dest(wb, name: str) -> Optional[Tuple[Worksheet, str]]:
    dn = wb.defined_names.get(name)
    if dn is None:
        return None
    try:
        dests = list(dn.destinations)
    except Exception:
        return None  # For constants or invalid named ranges

    for sheet_name, ref in dests:
        if sheet_name in wb.sheetnames:
            return wb[sheet_name], ref  # Return the sheet and reference like "A1:B3"
    return None

In [7]:
def shape(ref: str) -> Tuple[int, int]:
    min_c, min_r, max_c, max_r = range_boundaries(ref)
    return (max_r - min_r + 1, max_c - min_c + 1)

In [8]:
def read_values(ws: Worksheet, ref: str) -> List[List[object]]:
    min_c, min_r, max_c, max_r = range_boundaries(ref)
    out = []
    for row in ws.iter_rows(min_row=min_r, max_row=max_r, min_col=min_c, max_col=max_c, values_only=True):
        out.append(list(row))
    return out

In [9]:
def write_values(ws: Worksheet, ref: str, values: List[List[object]]) -> None:
    min_c, min_r, max_c, max_r = range_boundaries(ref)
    for r_i, row in enumerate(values):
        for c_i, v in enumerate(row):
            ws.cell(row=min_r + r_i, column=min_c + c_i, value=v)

In [44]:
def transfer_same_names(source_path: Path, target_path: Path, output_path: Path) -> None:
    print(f"Loading source: {source_path}")
    src = load_workbook(source_path, data_only=True, keep_links=False)

    print(f"Loading target template: {target_path}")
    tgt = load_workbook(target_path, data_only=False, keep_links=False)

    src_names = set(src.defined_names.keys())
    tgt_names = set(tgt.defined_names.keys())

    common = sorted(src_names & tgt_names)
    copied = 0

    for name in common:
        s = get_dest(src, name)
        t = get_dest(tgt, name)
        if not s or not t:
            print(f"[SKIP] {name}: not a cell range.")
            continue

        s_ws, s_ref = s
        t_ws, t_ref = t
        
        if shape(s_ref) != shape(t_ref):
            print(f"[SKIP] {name}: different shape.")
            continue

        vals = read_values(s_ws, s_ref)
        write_values(t_ws, t_ref, vals)
        print(f"[OK] {name} copied.")
        copied += 1

    print(f"\n✅ Done! Copied {copied} named ranges.")
    tgt.save(output_path)

In [17]:
from pathlib import Path

In [42]:
source_path = Path(r"C:\path\to\your\OLD_filled_workbook.xlsx")  # old template with data
output_path = Path(r"C:\path\to\output\VSME-filled.xlsx")        # where to save the result

# ✅ Your target template path (what you sent)
target_path = Path(r"\data_migration_try\Template\VSME-Digital-Template-Sample-1.0.1.xlsx")

In [30]:
import os
print(os.getcwd())

c:\Users\Thibault.magro\OneDrive - EFRAG\Desktop\Git repository\Paper team\PaperTeam\data_migration_try


In [45]:
transfer_same_names(
    Path(r"Template\VSME-Digital-Template-Sample-1.0.1.xlsx"),
    Path(r"Template\VSME-Digital-Template-1.1.1.xlsx"),
    Path(r"output.xlsx")
)

Loading source: Template\VSME-Digital-Template-Sample-1.0.1.xlsx
Loading target template: Template\VSME-Digital-Template-1.1.1.xlsx
[SKIP] AddressOfSite: different shape.
[SKIP] AmountOfEmissionToAir: different shape.
[OK] AmountOfEmissionToAir_unit copied.
[SKIP] AmountOfEmissionToSoil: different shape.
[OK] AmountOfEmissionToSoil_unit copied.
[SKIP] AmountOfEmissionToWater: different shape.
[OK] AmountOfEmissionToWater_unit copied.
[SKIP] AmountOfEmissionsTable: different shape.
[OK] AmountOfWaterWithdrawnAtSitesLocatedInAreasOfHighWaterStress copied.
[SKIP] AreaOfSiteInBiodiversitySensitiveArea: different shape.
[OK] AreaOfSiteInBiodiversitySensitiveArea_unit copied.
[OK] Assets copied.
[OK] AverageNumberOfAnnualTrainingHoursPerFemaleEmployee copied.
[OK] AverageNumberOfAnnualTrainingHoursPerMaleEmployee copied.
[OK] AverageNumberOfAnnualTrainingHoursPerNonReportedGenderEmployee copied.
[OK] AverageNumberOfAnnualTrainingHoursPerOtherGenderEmployee copied.
[OK] BaselineYearMember cop

In [36]:
import openpyxl, sys
print("openpyxl:", openpyxl.__version__, "python:", sys.version)

openpyxl: 3.1.5 python: 3.14.0 (tags/v3.14.0:ebf955d, Oct  7 2025, 10:15:03) [MSC v.1944 64 bit (AMD64)]
